## Agile Hardware Design
***
# FP Conclusion + Pattern Matching

<img src="./images/chisel_logo.svg" alt="agile hardware design logo" style="width:20%;float:right"/>

by Peter Hanping Chen based on 
1. UC Berkey bootcamp: Configuration file: source/load-ivy.sc
- https://github.com/freechipsproject/chisel-bootcamp
2. Prof. Scott Beamer, sbeamer@ucsc.edu
- [CSE 228A](https://classes.soe.ucsc.edu/cse228a/Winter24/)

## Plan for Today

* FP conclusion: flatMap, filter, sum
* Pattern matching
* Gracefully handling Option

## Loading The Chisel Library Into a Notebook

In [3]:
//interp.load.module(os.Path(s"${System.getProperty("user.dir")}/../resource/chisel_deps.sc"))
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
//val path = System.getProperty("user.dir") + "/source/chisel_deps.sc"
println("path: "+path)
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: /home/peter/AIU/AIU_CS800_Chisel/500_UCSC_HWD/012_FP_PatMatch/001_Code/source/load-ivy.sc


path: String = "/home/peter/AIU/AIU_CS800_Chisel/500_UCSC_HWD/012_FP_PatMatch/001_Code/source/load-ivy.sc"

In [4]:
import chisel3._
import chisel3.util._
import chiseltest._
import chiseltest.RawTester.test

import chisel3._

import chisel3.util._

import chiseltest._

import chiseltest.RawTester.test

## Scala `flatMap`

* Like `map`, but concatenates output of function's
  * More direct than calling `map` and then `flatten`
* Helpful for aggregating results of map when they are a collection
* Also useful if map will return 0 or more elements

In [6]:
val l = 0 until 5
print ("l: ")
println (l)
// Seq.fill(2)(3)
print ("l map { i => Seq.fill(i)(i) }: ")
println(l map { i => Seq.fill(i)(i) })
print ("(l map { i: Int => Seq.fill(i)(i) }).flatten: ")
println ((l map { i: Int => Seq.fill(i)(i) }).flatten)
print ("l flatMap { i => Seq.fill(i)(i) } : ")
println (l flatMap { i => Seq.fill(i)(i) })

// l flatMap { i => if (i%2 == 0) Seq(i) else Seq()}

l: Range 0 until 5
l map { i => Seq.fill(i)(i) }: Vector(List(), List(1), List(2, 2), List(3, 3, 3), List(4, 4, 4, 4))
(l map { i: Int => Seq.fill(i)(i) }).flatten: Vector(1, 2, 2, 3, 3, 3, 4, 4, 4, 4)
l flatMap { i => Seq.fill(i)(i) } : Vector(1, 2, 2, 3, 3, 3, 4, 4, 4, 4)


l: Range = Range(0, 1, 2, 3, 4)

## Visualizing `map` on Collections

<img src="images/mapOnC.svg" alt="map on collections" style="width:50%; align: left"/>

## Visualizing `flatMap`

<img src="images/flatmap.svg" alt="flatmap" style="width:50%;align:left"/>

## Applying Predicates in Scala

* A _predicate_ is a function that given a single element, returns a `Boolean`
* `filter` - elements persists to output collection only if predicate returns true
* `forall` - true if and only if predicate is true for all elements
* `exists` - true if predicate is true for at least one element

In [6]:
def isEven(x: Int): Boolean = x % 2 == 0
val l = 0 until 5

l filter isEven
// l filter { x => !isEven(x) }
l filterNot isEven
l forall isEven
l exists isEven

defined function isEven
l: Range = Range(0, 1, 2, 3, 4)
res5_2: collection.immutable.IndexedSeq[Int] = Vector(0, 2, 4)
res5_3: collection.immutable.IndexedSeq[Int] = Vector(1, 3)
res5_4: Boolean = false
res5_5: Boolean = true

## Visualizing `filter`

<img src="images/filter.svg" alt="filter viz" style="width:40%;align:left"/>

## Example: Prime Seive in Scala

In [7]:
def multipleOf(a: Int)(b: Int): Boolean = (b % a == 0)

def removeMultiplesOfX(l: Seq[Int], x: Int) = l filterNot multipleOf(x)

val allNums = 2 until 100

// println(allNums filterNot multipleOf(5))
// println(removeMultiplesOfX(allNums, 5))

def seive(s: Seq[Int]): Seq[Int] = {
    if (s.isEmpty) Seq()
    else Seq(s.head) ++ seive(removeMultiplesOfX(s.tail, s.head))
}

println(seive(allNums))

List(2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97)


defined function multipleOf
defined function removeMultiplesOfX
allNums: Range = Range(
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
...
defined function seive

## Scala Has Common Reductions Built-in

* `sum`, `product`, `min`, `max`

In [8]:
val l = 0 until 5
l reduce { _ + _ }
l.sum
l reduce { _ * _ }
l.product
l reduce { _ min _ }
l.min
l reduce { _ max _ }
l.max

l: Range = Range(0, 1, 2, 3, 4)
res7_1: Int = 10
res7_2: Int = 10
res7_3: Int = 0
res7_4: Int = 0
res7_5: Int = 0
res7_6: Int = 0
res7_7: Int = 4
res7_8: Int = 4

## Aside: Learn To Things the Scala Way

* In many cases, Scala provides methods for things other languages wouldn't, such as:
    * `isEmpty`/`nonEmpty`, `to`/`until`, `filter`/`filterNot`, `foldLeft`/`foldRight`
* _Problem:_ Language newcomers may not know about all of the features or common idioms
* _Solution A:_ Yet another reason to use an IDE, as it may recognize common launguage misuses
* _Solution B:_ Code reviews and looking at the code of others can help
* _Solution C:_ Read [Scala Collections Tips and Tricks](https://pavelfatin.com/scala-collections-tips-and-tricks/) by Pavel Fatin
    * Linked from course website under _Reference $\rightarrow$ Scala_

## Example: Using FP to do Matrix Multiplication (1/2)

In [9]:
// matrix in row-major layout
val mat = Seq.tabulate(4,4){ (i,j) => i+j }

def grabCol(m: Seq[Seq[Int]], i: Int) = m map { row => row(i) }
grabCol(mat,1)

def dotP(a: Seq[Int], b: Seq[Int]) = a.zip(b).map{ case (a_i,b_i) => a_i * b_i}.sum

def matMul(a: Seq[Seq[Int]], b: Seq[Seq[Int]]) = a map {
    rowOfA => (0 until rowOfA.size) map { colIndex => dotP(rowOfA, grabCol(b, colIndex)) }
}

matMul(mat, mat)

mat: Seq[Seq[Int]] = List(
  List(0, 1, 2, 3),
  List(1, 2, 3, 4),
  List(2, 3, 4, 5),
  List(3, 4, 5, 6)
)
defined function grabCol
res8_2: Seq[Int] = List(1, 2, 3, 4)
defined function dotP
defined function matMul
res8_5: Seq[collection.immutable.IndexedSeq[Int]] = List(
  Vector(14, 20, 26, 32),
  Vector(20, 30, 40, 50),
  Vector(26, 40, 54, 68),
  Vector(32, 50, 68, 86)
)

## Example: Using FP to do Matrix Multiplication (2/2)

In [10]:
def matMul(a: Seq[Seq[Int]], b: Seq[Seq[Int]]) = {
    (0 until a.size) map { i => {
        (0 until b.head.size) map {j => 
            dotP(a(i), grabCol(b, j))
        }
    }}
}

// def matMul(a: Seq[Seq[Int]], b: Seq[Seq[Int]]) = Seq.tabulate(a.length, b.head.length){
//     (i,j) => dotP(a(i), grabCol(b,j))
// }

matMul(mat, mat)

defined function matMul
res9_1: collection.immutable.IndexedSeq[collection.immutable.IndexedSeq[Int]] = Vector(
  Vector(14, 20, 26, 32),
  Vector(20, 30, 40, 50),
  Vector(26, 40, 54, 68),
  Vector(32, 50, 68, 86)
)

## Pattern Matching in Scala

* Can gracefully scale from replacing simple `if/else` and `switch` cases to more sophisticated searches
* Start block with `match` and list matches with `case`
* Can use `|` for or
* Can use `if` to specify condition
* Can use `_` for default (matched nothing above)

In [11]:
val x = 0

x match {
    case 0 => "0"
    case 1 | 3 => "nah"
    case y if (y%2 == 0) => "even"
    case 5 => "found it!"
    case _ => "other"
}

x: Int = 0
res10_1: String = "0"

## Can Match on Case Classes

* Can match on type as a whole, or even set fields

In [12]:
abstract class Vehicle

case class helicopter(color: String, driver: String) extends Vehicle

case class submarine(color: String, driver: String) extends Vehicle

val movers = Seq(helicopter("grey", "Marta"), helicopter("blue", "Laura"), submarine("yellow", "Paul"))

movers foreach {v => v match {
    case h: helicopter => println(s"${h.color} helicopter")
    case s: submarine => println(s"${s.color} submarine")
}}

movers foreach { _ match {
    case helicopter("blue", driver) => println(s"$driver has a blue helicopter")
    case s: submarine if (s.color != "yellow") => println(s"${s.driver}'s ${s.color} submarine")
    case _ => println("didn't match")
}}

grey helicopter
blue helicopter
yellow submarine
didn't match
Laura has a blue helicopter
didn't match


defined class Vehicle
defined class helicopter
defined class submarine
movers: Seq[Product with Serializable with Vehicle] = List(
  helicopter("grey", "Marta"),
  helicopter("blue", "Laura"),
  submarine("yellow", "Paul")
)

## More Graceful Interactions with `Option`

* Many Scala operations pass over None

In [13]:
val l = Seq.tabulate(5)(i => if (i % 2 == 1) Some(i) else None)

l foreach { x =>
    if (x.isDefined) println(x.get)
}

// l(1).getOrElse(-1)

// l.flatten

// l foreach { _ match {
//     case Some(i) => println(i)
//     case None => println("was empty")
// }}

// l(0) foreach println

1
3


l: Seq[Option[Int]] = List(None, Some(1), None, Some(3), None)

## Project Overview

### GOAL: gain experience developing/revising a generator

### Main Details
* Working in pairs (or individually)
* Pick an idea to build a generator for (contact instructor if need suggestion)
* Will _propose/design/develop/test/optimize/revise/document/present_ generator
* Ballpark for size/complexity: ~2x most recent homework problems
  * Building largely from scratch, so much more to do

## Project Timeline

* Weeks 5-6 (this week & next week) - find a partner and brainstorm ideas
    * Come chat in office hours to get early feedback
* Week 7 - propose project & get instructor feedback
* Week 8 - close the loop early and keep developing
* Week 9 - complete initial development & start revising
* Week 10 - finalize/polish project & present

## Project Deliverables

* 2/20 - initial proposal (<1 page)
  * What will generator do and what interface/parameters will it have?
  * Consider how to bootstrap, test, and what features that can be deferred
  * Feedback during 2/21 & 2/23
* 3/4 - Link to working repo (can be feature incomplete)
  * Close the loop early, and build from there
* 3/11 - External (peer) code review
* 3/13 or 3/15 - Presentation
* 3/20 - Links to final repo & revised presentation
  * Following presentation, will have time to make small revisions
  * Encouraged (but not required) to post publicly